In [1]:
import json
import igraph
import numpy as np

In [2]:
SG_train = json.load(open('train_sceneGraphs.json', 'r'))
SG_test = json.load(open('val_sceneGraphs.json', 'r'))

In [3]:
train_id = list(SG_train.keys())
test_id = list(SG_test.keys())

In [4]:
width, height = 0, 0
relations = []
objects = []
for idxImage in train_id:
    width = max(width, SG_train[idxImage]['width'])
    height = max(height, SG_train[idxImage]['height'])
    objects_list = list(SG_train[idxImage]['objects'].keys())
    for idxObj in objects_list:
        objects.append(SG_train[idxImage]['objects'][idxObj]['name'])
        for Relation in SG_train[idxImage]['objects'][idxObj]['relations']:
            relations.append(Relation['name'])

In [5]:
relations = list(dict.fromkeys(relations))
objects = list(dict.fromkeys(objects))

In [6]:
len(objects)

1702

In [7]:
objects[-1]

'printers'

In [8]:
len(relations)

310

In [9]:
width_test, height_test = 0, 0
relations_test = []
objects_test = []
for idxImage in test_id:
    width_test = max(width_test, SG_test[idxImage]['width'])
    height_test = max(height_test, SG_test[idxImage]['height'])
    objects_list = list(SG_test[idxImage]['objects'].keys())
    for idxObj in objects_list:
        objects_test.append(SG_test[idxImage]['objects'][idxObj]['name'])
        for Relation in SG_test[idxImage]['objects'][idxObj]['relations']:
            relations_test.append(Relation['name'])

In [10]:
width_test

1280

In [11]:
height_test

1280

In [12]:
relations_test = list(dict.fromkeys(relations_test))
objects_test = list(dict.fromkeys(objects_test))

In [13]:
len(objects_test)

1536

In [14]:
len(relations_test)

295

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from SG_VAE.word_embeddings import WordAE, WordDataset

In [16]:
losses = []
epochs = 100
batch_size = 256
dataset = WordDataset(objects)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
loss_function = nn.CrossEntropyLoss()
model = WordAE(objects, 100, 64, 3000)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
for epoch in range(epochs):
    total_loss = 0
    for batch_idx, sample in enumerate(loader):
        word_idxs = torch.tensor([model.encoder.word_to_ix[w] for w in sample], dtype=torch.long)
        model.zero_grad()
        pre_softmax = model(word_idxs)
        loss = loss_function(pre_softmax, word_idxs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
    print('epoch {},  loss: {}'.format(epoch, total_loss))

epoch 0,  loss: 53.518977642059326
epoch 1,  loss: 49.85872459411621
epoch 2,  loss: 42.13667821884155
epoch 3,  loss: 20.809243083000183
epoch 4,  loss: 1.3448963984847069
epoch 5,  loss: 0.11358659295365214
epoch 6,  loss: 0.008998108853120357
epoch 7,  loss: 0.014176919037709013
epoch 8,  loss: 0.0027763988182414323
epoch 9,  loss: 0.006829701931565069
epoch 10,  loss: 0.0010103382519446313
epoch 11,  loss: 0.000741654570447281
epoch 12,  loss: 0.000730853462300729
epoch 13,  loss: 0.0021812283448525704
epoch 14,  loss: 0.0004501321200223174
epoch 15,  loss: 0.0004080553808307741
epoch 16,  loss: 0.00036777979403268546
epoch 17,  loss: 0.0003398552871658467
epoch 18,  loss: 0.00032298134465236217
epoch 19,  loss: 0.00030406081350520253
epoch 20,  loss: 0.00028772813675459474
epoch 21,  loss: 0.0002686596635612659
epoch 22,  loss: 0.00026044814512715675
epoch 23,  loss: 0.0002472309715813026
epoch 24,  loss: 0.0002354961834498681
epoch 25,  loss: 0.00022587091007153504
epoch 26,  los

In [29]:
model.eval()
correct = 0
total = 0
objects_test.remove('pokemon')
test_dataset = WordDataset(objects_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
for sample in test_loader:
    word_idxs = torch.tensor([model.encoder.word_to_ix[w] for w in sample], dtype=torch.long)
    outputs = model(word_idxs)
    _, predicted = torch.max(outputs.data, 1)
    total += batch_size
    correct += (predicted == word_idxs).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 99 %
